## Домашнее задание `максимум 25 баллов (бывшее 10)`

## Критерии оценки 
`❗️Оцениваться будет значение метрики + ревью кода + реализация сервиса.` 

Вы можете сделать НЕ ВСЕ пункты и все равно получить 25 баллов. Получение > 25 баллов будет расцениваться как 25.


### 1. Побейте метрику на лидерборде `map@10 = 0.063` для userKnn модели с семинара (`4 балла`)


### 2. Предоставьте ноутбук(и) с экспериментами (`16 баллов`)

Что можно сделать:
   - (done) сделать кол-во рекомендаций не меньше N (`2 балла`)
   - (done) наличие тюнинга гиперпараметров (например, векторного расстояния или типов kNN моделей (implicit/rectools/...)) (`4 балла`)
   - (done) другие варианты ранжированивания айтемов похожих пользователей (`2 балла`)
   - (done) эксперименты с оффлайн валидацией (`2 балла`)
   - (done) в тесте вас ждут холодные пользователи. Сделайте рекомендации для них (обратите внимание на <a href="https://rectools.readthedocs.io/en/latest/api/rectools.models.popular.html"> rectools.models.popular</a>) (`2 балла`)
   - (done) блендинг моделей (`4 балла`)


### 3. Оберните модель в сервис.
- **предпочтительный онлайн вариант**: обучаете модель в ноутбуке, сохраняете обученную модель (pickle, dill), при запуске сервиса ее поднимаете и запрашиваете рекомендации "на лету" (`9 баллов`)
- или оффлайн вариант: предварительно посчитайте рекомендации для всех пользователей, сохраните и запрашивайте их (`4 балла`)
   

### Хороший код ДЗ это:
- комментарии и объяснения. В ipynb пользуйтесь силой маркдауна. 
В скриптах пишите комментарии и докстринг. 
- легкая читаемость и воспроизводимость
- стандарт PEP8 
- обоснование схемы валидации
- анализ метрики качества 


#Imports

In [3]:
pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 1.2 MB/s 


In [4]:
pip install rectools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 5.5 MB/s 
     |████████████████████████████████| 88 kB 8.9 MB/s 
     |████████████████████████████████| 1.1 MB 68.5 MB/s 
     |████████████████████████████████| 60 kB 6.6 MB/s 
     |████████████████████████████████| 13.4 MB 61.4 MB/s 
     |████████████████████████████████| 310 kB 12.9 MB/s 
     |████████████████████████████████| 188 kB 67.9 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp38-cp38-linux_x86_64.whl size=3825461 sha256=ff675d6c54eb4b3c15cdc89d644f966f150ebb6a6d0ec5a155cb7b167568fe71
  Stored in directory: /root/.cache/pip/wheels/00/ac/67/6f4536c819ed560c2c7e17c0f7a920e3e50c26108616087d05
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-linux_x86_64.whl size=738341 sha256=57d3e6bceb7e98fa8a640e538ae73f8099cd58fcbadc6bdfb3a4beeb00622ced
  Stored in directory: /root/.cache/pip/wheels/ec/bb/51/9c487d021

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import requests
from tqdm.auto import tqdm
from scipy.stats import mode 
from pprint import pprint
import dill
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender, BM25Recommender
import warnings
warnings.filterwarnings("ignore")

from rectools import Columns
from rectools.model_selection import TimeRangeSplitter
from rectools.dataset import Dataset, Interactions
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics, MAP
from rectools.models.popular import PopularModel
from collections import Counter


pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# Auxiliary functions

In [6]:
def get_coo_matrix(df, 
                   user_col='user_id', 
                   item_col='item_id', 
                   weight_col=None, 
                   users_mapping=None, 
                   items_mapping=None):
    if weight_col:
        weights = df[weight_col].astype(np.float32)
    else:
        weights = np.ones(len(df), dtype=np.float32)

    interaction_matrix = sp.sparse.coo_matrix((
        weights, 
        (
            df[user_col].map(users_mapping.get), 
            df[item_col].map(items_mapping.get)
        )
    ))
    return interaction_matrix

In [7]:
def generate_implicit_recs_mapper(model, N, users_mapping, users_inv_mapping):
    def _recs_mapper(user):
        user_id = users_mapping[user]
        recs = model.similar_items(user_id, N=N)
        return [users_inv_mapping[user] for user, _ in recs], [sim for _, sim in recs]
    return _recs_mapper

In [8]:
def idf_ranking(df: pd.DataFrame, train:pd.DataFrame):
    cnt = Counter(train['item_id'].values)
    idf = pd.DataFrame.from_dict(cnt, orient='index', columns=['doc_freq']).reset_index()
    n = train.shape[0]
    idf['idf'] = idf['doc_freq'].apply(lambda x: np.log((1 + n) / (1 + x) + 1))
    return df.merge(
        idf[['index', 'idf']], 
        left_on='item_id',
        right_on='index',
        how='left'
    ).drop(['index'], axis=1)

#Dataset loading

In [9]:
# Downloading dataset by chunks
url = "https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip"

req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download:   0%|          | 0.00/78.8M [00:00<?, ?iB/s]

In [10]:
!unzip kion_train.zip

Archive:  kion_train.zip
   creating: kion_train/
  inflating: kion_train/interactions.csv  
  inflating: __MACOSX/kion_train/._interactions.csv  
  inflating: kion_train/users.csv    
  inflating: __MACOSX/kion_train/._users.csv  
  inflating: kion_train/items.csv    
  inflating: __MACOSX/kion_train/._items.csv  


#EDA

In [14]:
interactions = pd.read_csv('kion_train/interactions.csv')
users = pd.read_csv('kion_train/users.csv')
items = pd.read_csv('kion_train/items.csv')

In [18]:
items.shape

(15963, 14)

In [15]:
interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
...,...,...,...,...,...
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0


In [12]:
# Renaming columns, converting timestamp
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight}, 
                    inplace=True) 

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

#1.UserKNN

##1.1.Train test split

In [13]:
# Train test split 
# Test = last 1 week 

n_folds = 1
unit = "W"
n_units = 1
periods = n_folds + 1
freq = f"{n_units}{unit}"

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)  
print(f"Start date and last date of the test fold: {start_date, last_date}")
    
date_range = pd.date_range(start=start_date, periods=periods, freq=freq, tz=last_date.tz)
print(f"Test fold borders: {date_range.values.astype('datetime64[D]')}")

# Generator of folds
cv = TimeRangeSplitter(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
print(f"Real number of folds: {cv.get_n_splits(Interactions(interactions))}")


Start date and last date of the test fold: (Timestamp('2021-08-08 00:00:00'), Timestamp('2021-08-22 00:00:00'))
Test fold borders: ['2021-08-08' '2021-08-15']
Real number of folds: 1


In [14]:
# We have just 1 test fold - no need to iterate over fold
(train_ids, test_ids, fold_info) = cv.split(Interactions(interactions), collect_fold_stats=True).__next__()

In [15]:
train_ids

array([      0,       1,       2, ..., 5476245, 5476247, 5476249])

In [16]:
test_ids

array([      6,      33,      56, ..., 5476229, 5476230, 5476240])

##1.2. Train matrix preparation



In [17]:
train = interactions.loc[train_ids]
test = interactions.loc[test_ids]

In [18]:
users_inv_mapping = dict(enumerate(train['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}

In [19]:
items_inv_mapping = dict(enumerate(train['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}

In [20]:
print(f"users_mapping amount: {len(users_mapping)}")
print(f"items_mapping amount: {len(items_mapping)}")

users_mapping amount: 842129
items_mapping amount: 15404


##1.3.Getting a sparse matrix

In [21]:
interaction_matrix = get_coo_matrix(train, weight_col='weight',
                                    users_mapping=users_mapping, 
                                    items_mapping=items_mapping)

In [22]:
interaction_matrix

<842129x15404 sparse matrix of type '<class 'numpy.float32'>'
	with 4587708 stored elements in COOrdinate format>

##1.4.Simple UserKNN model fitting

In [ ]:
userknn = CosineRecommender(K=10)
userknn.fit(interaction_matrix)

In [ ]:
# Saving the model
with open('userknn.dill', 'wb') as f:
    dill.dump(userknn, f)

In [116]:
with open('/content/drive/MyDrive/MTS_RecSys/userknn.dill', 'rb') as f:
    userknn = dill.load(f)

In [117]:
mapper = generate_implicit_recs_mapper(
    userknn, 
    N=N,
    users_mapping=users_mapping,
    users_inv_mapping=users_inv_mapping
)

In [118]:
recs = pd.DataFrame({
    'user_id': test['user_id'].unique()
})

recs['similar_user_id'], recs['similarity'] = zip(*recs['user_id'].map(mapper))
recs.head()

,user_id,similar_user_id,similarity
0,1016458,"[1016458, 416417, 859673, 494256, 414523, 254272, 101745, 50573, 55914, 589357]","[1.0000000098027124, 0.6143665279318272, 0.6115098337111928, 0.6031119652865972, 0.6016035410642744, 0.5956601420097666, 0.5951893645435953, 0.5910982730574769, 0.5901959801548787, 0.5899352526319..."
1,68478,"[68478, 20161, 412019, 620526, 695810, 879494, 589849, 988152, 691867, 594451]","[0.9999999999999997, 0.7739892962350301, 0.7739892916809104, 0.7739892904471841, 0.7739892904471841, 0.7739728844372993, 0.773732938463484, 0.7731990156224964, 0.764299456592446, 0.7371703228664362]"
2,580093,"[580093, 937473, 86046, 262791, 522603, 521456, 515818, 654291, 1009888, 333513]","[0.999999972768243, 0.98170707253408, 0.9816027812180024, 0.9811408740629928, 0.9809446394117618, 0.9804554967089238, 0.9787225812866681, 0.9780463490829185, 0.9761219827587835, 0.9753862067490225]"
3,1072552,"[1072552, 1022231, 449057, 639316, 38521, 992547, 401940, 613583, 184399, 33572]","[1.0000000074842705, 0.48938543442887494, 0.4845394132067774, 0.4778556198287305, 0.4747312631328243, 0.463755730200894, 0.46303331823589533, 0.4624616814044821, 0.46080084719982534, 0.45879293725..."
4,910002,"[910002, 490989, 57211, 914293, 442802, 185148, 594268, 737639, 57072, 55347]","[1.000000020041603, 0.7295340991209922, 0.7236776448102081, 0.7230887217258574, 0.722400619894437, 0.7218159408211805, 0.7216246604867983, 0.7181317479407374, 0.7177606301330373, 0.7175263799975921]"


In [119]:
# Exploding lists to get vertical representation
recs = recs.set_index('user_id').apply(pd.Series.explode).reset_index()

In [120]:
# Deleting recommendations of itself 
recs = recs[recs.user_id!=recs.similar_user_id]

In [121]:
watched = train.groupby('user_id').agg({'item_id': list})
watched.head()

,item_id
user_id,
0,"[7102, 14359, 15297, 6006, 9728, 12192]"
2,"[7571, 3541, 15266, 13867, 12841, 10770, 4475, 9506, 8936, 11018, 11577, 561, 7106, 6774, 16029, 8482, 6825, 3594, 16166, 5819, 2954, 383, 11689, 12449, 2025, 6155, 3628, 334, 4024, 7210, 11539, 1..."
3,"[9728, 16406, 10440, 3475, 4151, 1418, 2220, 3734, 13789, 8581, 13849, 9550, 10464, 4880, 4436, 8801, 11790, 2657, 8252]"
4,"[4700, 6317]"
5,"[14397, 6445, 11437, 5651, 6167, 12466, 632, 8450, 2685, 7825, 3145, 7043, 4179, 15890, 5115, 4719, 10848]"


In [122]:
# Joining watched items
recs = recs.merge(watched, left_on=['similar_user_id'], right_on=['user_id'], how='left')
recs = recs.explode('item_id')

In [123]:
# Dropping duplicates pairs user_id-item_id and keeping rows with the highest similiarity

recs = recs.sort_values(['user_id', 'similarity'], ascending=False)

In [124]:
recs = recs.drop_duplicates(['user_id', 'item_id'], keep='first')

##1.5.Using idf for ranking

In [125]:
recs = idf_ranking(recs, train)

In [126]:
recs['rank_idf'] = recs['similarity'] * recs['idf']

In [127]:
# Making order by rank_idf 
recs = recs.sort_values(['user_id', 'rank_idf'], ascending=False)

In [128]:
# Making rank
recs['rank'] = recs.groupby('user_id').cumcount() + 1 

In [129]:
# Decreasing rank to N
recs = recs[recs['rank'] <= N]

In [130]:
recs

,user_id,similar_user_id,similarity,item_id,idf,rank_idf,rank
1,1097544,412291,0.704641,7829,5.770857,4.066384,1
0,1097544,412291,0.704641,8636,4.958912,3.494255,2
2,1097544,412291,0.704641,9996,4.928260,3.472656,3
16,1097534,1081871,0.942736,8586,9.870883,9.305635,1
11,1097534,331384,0.951861,475,8.081493,7.692454,2
...,...,...,...,...,...,...,...
1813671,3,325741,0.906174,14603,7.527456,6.821187,6
1813659,3,97015,0.908573,3768,7.475807,6.792314,7
1813670,3,325741,0.906174,12259,7.472355,6.771256,8
1813660,3,97015,0.908573,6089,7.271497,6.606683,9


##1.6.Dealing with less than N recommendations

In [131]:
# We have users with less than N recommendations
s = recs['user_id'].value_counts()
s[s<N]

197452     9
615926     9
1058629    9
501864     9
928027     9
          ..
204717     1
991217     1
204767     1
850663     1
361516     1
Name: user_id, Length: 48670, dtype: int64

In [132]:
recs[recs['user_id'].isin(s[s<N].index)]

,user_id,similar_user_id,similarity,item_id,idf,rank_idf,rank
1,1097544,412291,0.704641,7829,5.770857,4.066384,1
0,1097544,412291,0.704641,8636,4.958912,3.494255,2
2,1097544,412291,0.704641,9996,4.928260,3.472656,3
63,1097512,984690,1.0,3734,4.286431,4.28643,1
64,1097512,553951,1.0,15297,3.352498,3.352497,2
...,...,...,...,...,...,...,...
1813491,119,123470,0.996359,7626,5.930396,5.908801,2
1813490,119,591119,0.999797,4436,5.667337,5.666187,3
1813489,119,591119,0.999797,9728,3.792339,3.79157,4
1813493,107,507982,1.0,2657,4.307105,4.307105,1


In [137]:
not_enough_dict = recs[recs['user_id'].isin(s[s<N].index)].groupby('user_id')['item_id'].agg(lambda row: list(row)).to_dict()

### Popular model

In [67]:
# Creating dataset for PopularModel
dataset_train = Dataset.construct(
    interactions_df=train,
    user_features_df=None,
    item_features_df=None
)

In [68]:
popular_model = PopularModel()
popular_model.fit(dataset_train)

In [69]:
popular_model_result = popular_model.recommend(
    dataset_train.user_id_map.external_ids[:1], 
    dataset=dataset_train, 
    k=2*N, 
    filter_viewed=False,
)['item_id'].to_list()

In [70]:
popular_model_result

[10440,
 15297,
 13865,
 9728,
 4151,
 3734,
 2657,
 4880,
 142,
 6809,
 9996,
 4740,
 8636,
 12192,
 11237,
 7571,
 1844,
 12995,
 14741,
 14431]

In [143]:
d = {"user_id":[], "item_id": [], "rank": []}
# for user_id, max_rank in not_enough_dict.items():
for user_id, item_id_list in not_enough_dict.items():
    max_rank = len(item_id_list)
    # drop popular items, that is already in recommendation list
    corrected_popular_list = [item for item in popular_model_result if item not in item_id_list]
    for i in range(1, N - max_rank + 1):
        d['user_id'].append(user_id)
        d['item_id'].append(corrected_popular_list[i])
        d['rank'].append(max_rank+i)

In [145]:
filled_with_popular_userknn = pd.DataFrame.from_dict(d)

In [150]:
final_result_userknn = pd.concat([recs,filled_with_popular_userknn])

In [151]:
s = final_result_userknn['user_id'].value_counts()
s[s<10]

Series([], Name: user_id, dtype: int64)

In [152]:
with open('/content/drive/MyDrive/MTS_RecSys/final_result_userknn.dill', 'wb') as f:
    dill.dump(final_result_userknn, f)

##1.7.Metrics

In [26]:
# Calculating several metrics, including MAP@10
metrics = {
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
    "map@10": MAP(k=10),
}

In [27]:
catalog = train['item_id'].unique()

In [179]:
metric_values = calc_metrics(
      metrics,
      reco=final_result_userknn,
      interactions=test,
      prev_interactions=train,
      catalog=catalog
)

In [180]:
metric_values

{'prec@10': 0.015735955992665446,
 'recall@10': 0.08167169516943629,
 'map@10': 0.020269051400935267,
 'novelty': 7.168520634937293,
 'serendipity': 5.6443609141752035e-05}

#2.Experiments

##2.1.Other ranking approaches

### TF-IDF

In [ ]:
tfidf = TFIDFRecommender(K=10)
tfidf.fit(interaction_matrix)

  0%|          | 0/842129 [00:00<?, ?it/s]

In [ ]:
# with open('/content/drive/MyDrive/MTS_RecSys/tfidf.dill', 'wb') as f:
#     dill.dump(tfidf, f)

In [153]:
with open('/content/drive/MyDrive/MTS_RecSys/tfidf.dill', 'rb') as f:
    tfidf = dill.load(f)

In [154]:
mapper = generate_implicit_recs_mapper(
    tfidf, 
    N=N,
    users_mapping=users_mapping,
    users_inv_mapping=users_inv_mapping
)

In [155]:
recs = pd.DataFrame({
    'user_id': test['user_id'].unique()
})

recs['similar_user_id'], recs['similarity'] = zip(*recs['user_id'].map(mapper))
recs.head()

,user_id,similar_user_id,similarity
0,1016458,"[1016458, 542443, 983874, 263122, 1087135, 299198, 416417, 107556, 980255, 55914]","[0.9999999999999997, 0.30795056944195065, 0.2892556288628258, 0.2865786318030271, 0.2840258603538891, 0.28275509313396374, 0.2814949055823506, 0.2806520562356141, 0.27569737451648724, 0.2703263699..."
1,68478,"[68478, 620526, 695810, 20161, 412019, 879494, 589849, 988152, 691867, 1059229]","[0.9999999999999998, 0.6879105673985805, 0.6879105673985805, 0.6879105673985805, 0.6879105673985805, 0.6875684365910972, 0.682527643497476, 0.6810886673793897, 0.6649011119333184, 0.6500976114158797]"
2,580093,"[580093, 805265, 980409, 939663, 708439, 736490, 943049, 86046, 522603, 521456]","[1.0, 0.7937798109102163, 0.788744014339269, 0.7666797469287155, 0.7631606741401563, 0.750189495128039, 0.7501195711404515, 0.7474499190666032, 0.7473880080188159, 0.7471532852053147]"
3,1072552,"[1072552, 162216, 613583, 896269, 449057, 246357, 992547, 49193, 503934, 639316]","[1.0000000000000002, 0.45699036174245683, 0.45472129140483386, 0.4278037012555298, 0.42654520594852924, 0.42378450328959727, 0.41383525556374756, 0.411008423568197, 0.4078683617582729, 0.407690880..."
4,910002,"[910002, 301452, 454746, 15649, 25643, 90163, 8179, 43300, 472743, 590422]","[0.9999999999999997, 0.5504875268884805, 0.5503635062149785, 0.5497829629361086, 0.5497829629361086, 0.5497829629361086, 0.5497829629361086, 0.5497829629361086, 0.5497829629361086, 0.5497829629361..."


In [156]:
# Exploding lists to get vertical representation
recs = recs.set_index('user_id').apply(pd.Series.explode).reset_index()

In [157]:
# Deleting recommendations of itself 
recs = recs[recs.user_id!=recs.similar_user_id]

In [158]:
watched = train.groupby('user_id').agg({'item_id': list})
watched.head()

,item_id
user_id,
0,"[7102, 14359, 15297, 6006, 9728, 12192]"
2,"[7571, 3541, 15266, 13867, 12841, 10770, 4475, 9506, 8936, 11018, 11577, 561, 7106, 6774, 16029, 8482, 6825, 3594, 16166, 5819, 2954, 383, 11689, 12449, 2025, 6155, 3628, 334, 4024, 7210, 11539, 1..."
3,"[9728, 16406, 10440, 3475, 4151, 1418, 2220, 3734, 13789, 8581, 13849, 9550, 10464, 4880, 4436, 8801, 11790, 2657, 8252]"
4,"[4700, 6317]"
5,"[14397, 6445, 11437, 5651, 6167, 12466, 632, 8450, 2685, 7825, 3145, 7043, 4179, 15890, 5115, 4719, 10848]"


In [159]:
# Joining watched items
recs = recs.merge(watched, left_on=['similar_user_id'], right_on=['user_id'], how='left')
recs = recs.explode('item_id')

In [160]:
# Dropping duplicates pairs user_id-item_id and keeping rows with the highest similiarity
recs = recs.sort_values(['user_id', 'similarity'], ascending=False)

In [161]:
recs = recs.drop_duplicates(['user_id', 'item_id'], keep='first')

In [162]:
# Making rank
recs['rank'] = recs.groupby('user_id').cumcount() + 1 

In [163]:
# Decreasing rank to N
recs = recs[recs['rank'] <=N]

#### Dealing with less than N recommendations

In [164]:
# We have users with less than N recommendations
s = recs['user_id'].value_counts()
s[s<N]

556008    9
365738    9
137343    9
553069    9
231048    9
         ..
389271    1
389317    1
389370    1
389378    1
387893    1
Name: user_id, Length: 58932, dtype: int64

In [165]:
recs[recs['user_id'].isin(s[s<N].index)]

,user_id,similar_user_id,similarity,item_id,rank
99507,1097512,984690,0.999999,10440,1
99507,1097512,984690,0.999999,3734,2
603965,1097479,392250,0.849606,3734,1
603965,1097479,392250,0.849606,10770,2
603966,1097479,185082,0.838868,13865,3
...,...,...,...,...,...
849408,98,1002728,0.522362,10724,3
849409,98,978382,0.489714,4946,4
849409,98,978382,0.489714,4740,5
849410,98,436269,0.488654,2909,6


In [166]:
not_enough_dict = recs[recs['user_id'].isin(s[s<N].index)].groupby('user_id')['item_id'].agg(lambda row: list(row)).to_dict()

In [167]:
# Creating dataset for PopularModel
dataset_train = Dataset.construct(
    interactions_df=train,
    user_features_df=None,
    item_features_df=None
)

In [168]:
popular_model = PopularModel()
popular_model.fit(dataset_train)

In [169]:
popular_model_result = popular_model.recommend(
    dataset_train.user_id_map.external_ids[:1], 
    dataset=dataset_train, 
    k=2*N, 
    filter_viewed=False,
)['item_id'].to_list()

In [170]:
popular_model_result

[10440,
 15297,
 13865,
 9728,
 4151,
 3734,
 2657,
 4880,
 142,
 6809,
 9996,
 4740,
 8636,
 12192,
 11237,
 7571,
 1844,
 12995,
 14741,
 14431]

In [171]:
d = {"user_id":[], "item_id": [], "rank": []}
# for user_id, max_rank in not_enough_dict.items():
for user_id, item_id_list in not_enough_dict.items():
    max_rank = len(item_id_list)
    # drop popular items, that is already in recommendation list
    corrected_popular_list = [item for item in popular_model_result if item not in item_id_list]
    for i in range(1, N - max_rank + 1):
        d['user_id'].append(user_id)
        d['item_id'].append(corrected_popular_list[i])
        d['rank'].append(max_rank+i)

In [172]:
filled_with_popular_tfidf = pd.DataFrame.from_dict(d)

In [173]:
final_result_tfidf = pd.concat([recs,filled_with_popular_tfidf])

In [174]:
s = final_result_tfidf['user_id'].value_counts()
s[s<10]

Series([], Name: user_id, dtype: int64)

In [175]:
with open('/content/drive/MyDrive/MTS_RecSys/final_result_tfidf.dill', 'wb') as f:
    dill.dump(final_result_tfidf, f)

In [176]:
catalog = train['item_id'].unique()

In [177]:
tfidf_metric_values = calc_metrics(
      metrics,
      reco=final_result_tfidf,
      interactions=test,
      prev_interactions=train,
      catalog=catalog
)

In [178]:
tfidf_metric_values

{'prec@10': 0.020649520018042225,
 'recall@10': 0.10676722963456782,
 'map@10': 0.02652848400918433,
 'novelty': 5.553259996354108,
 'serendipity': 3.5661279648735724e-05}

### BM25

In [ ]:
bm25 = BM25Recommender(K=10)
bm25.fit(interaction_matrix)

  0%|          | 0/842129 [00:00<?, ?it/s]

In [ ]:
# with open('/content/drive/MyDrive/MTS_RecSys/bm25.dill', 'wb') as f:
#     dill.dump(bm25, f)

In [181]:
with open('/content/drive/MyDrive/MTS_RecSys/bm25.dill', 'rb') as f:
    bm25 = dill.load(f)

In [182]:
mapper = generate_implicit_recs_mapper(
    bm25, 
    N=N,
    users_mapping=users_mapping,
    users_inv_mapping=users_inv_mapping
)

In [183]:
recs = pd.DataFrame({
    'user_id': test['user_id'].unique()
})

recs['similar_user_id'], recs['similarity'] = zip(*recs['user_id'].map(mapper))
recs.head()

,user_id,similar_user_id,similarity
0,1016458,"[1016458, 768373, 409259, 978218, 474785, 214708, 424619, 237669, 939159, 994484]","[6499.893803235837, 2487.573564889936, 2065.294335056916, 2004.3882317070374, 1960.3753230886257, 1896.5537519669006, 1757.7864748694478, 1733.6649020797067, 1720.1064215978429, 1717.7063205770794]"
1,68478,"[68478, 1090577, 557478, 21325, 1035803, 414535, 758605, 76520, 222973, 225261]","[1903.1537667306072, 640.3587502340363, 635.114972805723, 616.4652575942256, 581.0693350135408, 559.5973779271835, 558.0152672009923, 557.9529612605429, 557.7081973209919, 557.3893362356985]"
2,580093,"[580093, 110930, 721287, 764504, 416206, 456600, 6809, 483830, 691773, 164699]","[1269.3857345818258, 873.5346833467504, 657.9231090698622, 646.8836638907121, 634.2838455066677, 616.8245114660205, 596.7809781224956, 582.1506277248486, 576.5840739693372, 570.9506930907593]"
3,1072552,"[1072552, 280117, 359685, 613264, 902950, 424970, 706721, 923225, 977188, 297838]","[8375.282512507674, 4212.727992560902, 3867.0859213287026, 3568.5926452438302, 3551.2888440414063, 3403.626381400169, 3352.9232868087497, 3026.6220760411707, 2937.0868759436635, 2808.5133335752375]"
4,910002,"[910002, 376977, 126581, 720475, 173180, 863601, 779856, 861848, 933021, 150249]","[3744.88772030505, 1729.6186535669642, 1648.2044184296262, 1617.328874342526, 1536.7342845343237, 1532.073197555375, 1526.6357901274998, 1501.2448283809945, 1498.0611495961562, 1496.9186202603807]"


#### Batch processing

In [184]:
%%time
# Batch processing, because of the large amount data in the intersection with items and RAM failure

N = 10
watched = train.groupby('user_id').agg({'item_id': list})

df = recs
chunks = []
for chunk in np.array_split(df, 5):
    # Exploding lists to get vertical representation
    chunk = chunk.set_index('user_id').apply(pd.Series.explode).reset_index()

    # Deleting recommendations of itself 
    chunk = chunk[chunk.user_id!=chunk.similar_user_id]

    # Joining watched items
    chunk = chunk.merge(watched, left_on=['similar_user_id'], right_on=['user_id'], how='left')
    chunk = chunk.explode('item_id')

    # Dropping duplicates pairs user_id-item_id and keeping rows with the highest similiarity
    chunk = chunk.sort_values(['user_id', 'similarity'], ascending=False)
    chunk = chunk.drop_duplicates(['user_id', 'item_id'], keep='first')

    # Making rank
    chunk['rank'] = chunk.groupby('user_id').cumcount() + 1 

    # Selecting top-N
    chunk = chunk[chunk['rank'] <= N]

    # For future concat
    chunks.append(chunk)

CPU times: user 1min 43s, sys: 5.57 s, total: 1min 49s
Wall time: 2min 3s


In [192]:
chunk

,user_id,similar_user_id,similarity,item_id,rank
131028,1097544,153429,728.748166,3682,1
131028,1097544,153429,728.748166,14317,2
131028,1097544,153429,728.748166,4065,3
131028,1097544,153429,728.748166,5873,4
131028,1097544,153429,728.748166,4621,5
...,...,...,...,...,...
128802,60,424619,1189.930878,5770,6
128802,60,424619,1189.930878,4482,7
128802,60,424619,1189.930878,3066,8
128802,60,424619,1189.930878,5910,9


In [198]:
# Union dataframe back
result_df = pd.concat(chunks)

# Dropping 'similar_user_id' and 'similarity' columns
result_df = result_df.drop(columns=['similar_user_id','similarity'])

In [200]:
result_df

,user_id,item_id,rank
100656,1097512,12995,1
100656,1097512,10440,2
100656,1097512,3734,3
100656,1097512,4880,4
100657,1097512,12192,5
...,...,...,...
128802,60,5770,6
128802,60,4482,7
128802,60,3066,8
128802,60,5910,9


#### Dealing with less than N recommendations

In [201]:
recs = result_df

In [202]:
# We have users with less than N recommendations
s = recs['user_id'].value_counts()
s[s<N]

536316     9
266343     9
798109     9
159391     9
896379     9
          ..
348081     1
232427     1
1063305    1
965220     1
234348     1
Name: user_id, Length: 7329, dtype: int64

In [203]:
recs[recs['user_id'].isin(s[s<N].index)]

,user_id,item_id,rank
21281,1096776,831,1
21281,1096776,11863,2
21281,1096776,6253,3
21283,1096776,10440,4
21283,1096776,9996,5
...,...,...,...
17476,1885,10440,3
17476,1885,13865,4
39981,1610,12192,1
39988,1610,8636,2


In [204]:
recs

,user_id,item_id,rank
100656,1097512,12995,1
100656,1097512,10440,2
100656,1097512,3734,3
100656,1097512,4880,4
100657,1097512,12192,5
...,...,...,...
128802,60,5770,6
128802,60,4482,7
128802,60,3066,8
128802,60,5910,9


In [205]:
not_enough_dict = recs[recs['user_id'].isin(s[s<N].index)].groupby('user_id')['item_id'].agg(lambda row: list(row)).to_dict()

In [206]:
# Creating dataset for PopularModel
dataset_train = Dataset.construct(
    interactions_df=train,
    user_features_df=None,
    item_features_df=None
)

In [207]:
popular_model = PopularModel()
popular_model.fit(dataset_train)

In [208]:
popular_model_result = popular_model.recommend(
    dataset_train.user_id_map.external_ids[:1], 
    dataset=dataset_train, 
    k=2*N, 
    filter_viewed=False,
)['item_id'].to_list()

In [209]:
d = {"user_id":[], "item_id": [], "rank": []}
# for user_id, max_rank in not_enough_dict.items():
for user_id, item_id_list in not_enough_dict.items():
    max_rank = len(item_id_list)
    # drop popular items, that is already in recommendation list
    corrected_popular_list = [item for item in popular_model_result if item not in item_id_list]
    for i in range(1, N - max_rank + 1):
        d['user_id'].append(user_id)
        d['item_id'].append(corrected_popular_list[i])
        d['rank'].append(max_rank+i)

In [210]:
filled_with_popular_bm_25 = pd.DataFrame.from_dict(d)

In [211]:
final_result_bm_25 = pd.concat([recs,filled_with_popular_bm_25])

In [212]:
s = final_result_bm_25['user_id'].value_counts()
s[s<10]

Series([], Name: user_id, dtype: int64)

In [213]:
with open('/content/drive/MyDrive/MTS_RecSys/final_result_bm_25.dill', 'wb') as f:
    dill.dump(final_result_bm_25, f)

In [214]:
bm_25_metric_values = calc_metrics(
  metrics,
  reco=final_result_bm_25,
  interactions=test,
  prev_interactions=train,
  catalog=catalog
)

In [216]:
bm_25_metric_values

{'prec@10': 0.010149730837492524,
 'recall@10': 0.05202168689019393,
 'map@10': 0.014336336556977794,
 'novelty': 8.558863847296637,
 'serendipity': 7.578220852014551e-05}

In [217]:
metric_values

{'prec@10': 0.015735955992665446,
 'recall@10': 0.08167169516943629,
 'map@10': 0.020269051400935267,
 'novelty': 7.168520634937293,
 'serendipity': 5.6443609141752035e-05}

In [218]:
tfidf_metric_values

{'prec@10': 0.020649520018042225,
 'recall@10': 0.10676722963456782,
 'map@10': 0.02652848400918433,
 'novelty': 5.553259996354108,
 'serendipity': 3.5661279648735724e-05}

##2.2.Hyperparameter tuning

Since models CosineRecommender and TFIDFRecommender have the best performance in terms of metrics, then we will try to improve the results by tuning the K parameter.

K : The number of neighbours to include when calculating the item-item similarity matrix.

In [63]:
models = {
    'CR_10': CosineRecommender(K=10),
    'CR_15': CosineRecommender(K=15),
    'CR_20': CosineRecommender(K=20),
    'TFIDF_10' : TFIDFRecommender(K=10),
    'TFIDF_15' : TFIDFRecommender(K=15),
    'TFIDF_20' : TFIDFRecommender(K=20)
}

In [ ]:
# # Training models with different parameters, then saving them
# models_dir = '/content/drive/MyDrive/MTS_RecSys'

# for model_name, model in models.items():
#     model.fit(interaction_matrix)
#     # Saving model
#     with open(f'{models_dir}/{model_name}.dill', 'wb') as f:
#         dill.dump(model, f)


In [64]:
# Loading trained models
models_dir = ('/content/drive/MyDrive/MTS_RecSys')

for model_name in models.keys():
    with open(f'{models_dir}/{model_name}.dill', 'rb') as f:
        models[model_name] = dill.load(f)

In [65]:
models

{'CR_10': <implicit.nearest_neighbours.CosineRecommender at 0x7f796991b340>,
 'CR_15': <implicit.nearest_neighbours.CosineRecommender at 0x7f796161d1c0>,
 'CR_20': <implicit.nearest_neighbours.CosineRecommender at 0x7f796161dac0>,
 'TFIDF_10': <implicit.nearest_neighbours.TFIDFRecommender at 0x7f79540261c0>,
 'TFIDF_15': <implicit.nearest_neighbours.TFIDFRecommender at 0x7f79540260d0>,
 'TFIDF_20': <implicit.nearest_neighbours.TFIDFRecommender at 0x7f79540262b0>}

### main

In [71]:
results = []
N = 10

for model_name, model in models.items():
    print('Starting with:', model_name)
    mapper = generate_implicit_recs_mapper(
        model, 
        N=N,
        users_mapping=users_mapping,
        users_inv_mapping=users_inv_mapping
    )
    recs = pd.DataFrame({
        'user_id': test['user_id'].unique()
    })
    recs['similar_user_id'], recs['similarity'] = zip(*recs['user_id'].map(mapper))

    # Exploding lists to get vertical representation
    recs = recs.set_index('user_id').apply(pd.Series.explode).reset_index()
    # Deleting recommendations of itself   
    recs = recs[~(recs['user_id']==recs['similar_user_id'])]

    # Joining watched items
    watched = train.groupby('user_id').agg({'item_id': list})
    recs = recs.merge(watched, left_on=['similar_user_id'], right_on=['user_id'], how='left')
    recs = recs.explode('item_id')

    # Dropping duplicates pairs user_id-item_id and keeping rows with the highest similiarity
    recs = recs.sort_values(['user_id', 'similarity'], ascending=False)
    recs = recs.drop_duplicates(['user_id', 'item_id'], keep='first')

    # Making rank
    recs['rank'] = recs.groupby('user_id').cumcount() + 1 

    # Decreasing rank to 10
    recs = recs[recs['rank'] <= 10]

    # Dealing witn less than N recommendations
    s = recs['user_id'].value_counts()
    not_enough_dict = recs[recs['user_id'].isin(s[s<N].index)].groupby('user_id')['item_id'].agg(lambda row: list(row)).to_dict()

    d = {"user_id":[], "item_id": [], "rank": []}
    # for user_id, max_rank in not_enough_dict.items():
    for user_id, item_id_list in not_enough_dict.items():
        max_rank = len(item_id_list)
        # drop popular items, that is already in recommendation list
        corrected_popular_list = [item for item in popular_model_result if item not in item_id_list]
        for i in range(1, N - max_rank + 1):
            d['user_id'].append(user_id)
            d['item_id'].append(corrected_popular_list[i])
            d['rank'].append(max_rank+i)

    filled_with_popular = pd.DataFrame.from_dict(d)
    final_result = pd.concat([recs,filled_with_popular])

    model_result = {'name': model_name}
    metric_values = calc_metrics(
                metrics,
                reco=final_result, 
                interactions=test,
                prev_interactions=train,
                catalog=catalog
            )
    model_result.update(metric_values)
    results.append(model_result)

Starting with: CR_10
Starting with: CR_15
Starting with: CR_20
Starting with: TFIDF_10
Starting with: TFIDF_15
Starting with: TFIDF_20


### Tuning_results

In [72]:
tuning_results = pd.DataFrame.from_dict(results)

In [73]:
tuning_results

,name,prec@10,recall@10,map@10,novelty,serendipity
0,CR_10,0.018499,0.094935,0.023819,5.795980,0.000038
1,CR_15,0.018498,0.094932,0.023819,5.796153,0.000038
2,CR_20,0.018499,0.094935,0.023821,5.796000,0.000038
3,TFIDF_10,0.020650,0.106767,0.026528,5.553260,0.000036
4,TFIDF_15,0.020650,0.106767,0.026528,5.553260,0.000036
5,TFIDF_20,0.020650,0.106767,0.026528,5.553260,0.000036


Let's take CR_20 and TFIDF_15 and **blend** them. They have best results across their model family

In [75]:
with open('/content/drive/MyDrive/MTS_RecSys/CR_20.dill', 'rb') as f:
    CR_20 = dill.load(f)

In [79]:
with open('/content/drive/MyDrive/MTS_RecSys/TFIDF_15.dill', 'rb') as f:
    TFIDF_15 = dill.load(f)

#### CR_20

In [82]:
mapper = generate_implicit_recs_mapper(
    CR_20, 
    N=N,
    users_mapping=users_mapping,
    users_inv_mapping=users_inv_mapping
)

In [83]:
recs = pd.DataFrame({
    'user_id': test['user_id'].unique()
})

recs['similar_user_id'], recs['similarity'] = zip(*recs['user_id'].map(mapper))
recs.head()

,user_id,similar_user_id,similarity
0,1016458,"[1016458, 416417, 859673, 494256, 414523, 254272, 101745, 50573, 55914, 589357]","[1.0000000098027124, 0.6143665279318272, 0.6115098337111928, 0.6031119652865972, 0.6016035410642744, 0.5956601420097666, 0.5951893645435953, 0.5910982730574769, 0.5901959801548787, 0.5899352526319..."
1,68478,"[68478, 20161, 412019, 620526, 695810, 879494, 589849, 988152, 691867, 594451]","[0.9999999999999997, 0.7739892962350301, 0.7739892916809104, 0.7739892904471841, 0.7739892904471841, 0.7739728844372993, 0.773732938463484, 0.7731990156224964, 0.764299456592446, 0.7371703228664362]"
2,580093,"[580093, 937473, 86046, 262791, 522603, 521456, 515818, 654291, 1009888, 333513]","[0.999999972768243, 0.98170707253408, 0.9816027812180024, 0.9811408740629928, 0.9809446394117618, 0.9804554967089238, 0.9787225812866681, 0.9780463490829185, 0.9761219827587835, 0.9753862067490225]"
3,1072552,"[1072552, 1022231, 449057, 639316, 38521, 992547, 401940, 613583, 184399, 33572]","[1.0000000074842705, 0.48938543442887494, 0.4845394132067774, 0.4778556198287305, 0.4747312631328243, 0.463755730200894, 0.46303331823589533, 0.4624616814044821, 0.46080084719982534, 0.45879293725..."
4,910002,"[910002, 490989, 57211, 914293, 442802, 185148, 594268, 737639, 57072, 55347]","[1.000000020041603, 0.7295340991209922, 0.7236776448102081, 0.7230887217258574, 0.722400619894437, 0.7218159408211805, 0.7216246604867983, 0.7181317479407374, 0.7177606301330373, 0.7175263799975921]"


In [84]:
# Exploding lists to get vertical representation
recs = recs.set_index('user_id').apply(pd.Series.explode).reset_index()

In [85]:
# Deleting recommendations of itself 
recs = recs[recs.user_id!=recs.similar_user_id]

In [86]:
watched = train.groupby('user_id').agg({'item_id': list})
watched.head()

,item_id
user_id,
0,"[7102, 14359, 15297, 6006, 9728, 12192]"
2,"[7571, 3541, 15266, 13867, 12841, 10770, 4475, 9506, 8936, 11018, 11577, 561, 7106, 6774, 16029, 8482, 6825, 3594, 16166, 5819, 2954, 383, 11689, 12449, 2025, 6155, 3628, 334, 4024, 7210, 11539, 1..."
3,"[9728, 16406, 10440, 3475, 4151, 1418, 2220, 3734, 13789, 8581, 13849, 9550, 10464, 4880, 4436, 8801, 11790, 2657, 8252]"
4,"[4700, 6317]"
5,"[14397, 6445, 11437, 5651, 6167, 12466, 632, 8450, 2685, 7825, 3145, 7043, 4179, 15890, 5115, 4719, 10848]"


In [87]:
# Joining watched items
recs = recs.merge(watched, left_on=['similar_user_id'], right_on=['user_id'], how='left')
recs = recs.explode('item_id')

In [88]:
# Dropping duplicates pairs user_id-item_id and keeping rows with the highest similiarity
recs = recs.sort_values(['user_id', 'similarity'], ascending=False)

In [89]:
recs = recs.drop_duplicates(['user_id', 'item_id'], keep='first')

In [90]:
# Making rank
recs['rank'] = recs.groupby('user_id').cumcount() + 1 

In [91]:
# Decreasing rank to N
recs = recs[recs['rank'] <=N]

In [92]:
# We have users with less than N recommendations
s = recs['user_id'].value_counts()
s[s<N]

870236    9
837761    9
917889    9
976662    9
233601    9
         ..
204767    1
204789    1
204794    1
204799    1
361516    1
Name: user_id, Length: 48670, dtype: int64

In [93]:
recs[recs['user_id'].isin(s[s<N].index)]

,user_id,similar_user_id,similarity,item_id,rank
872642,1097544,412291,0.704641,8636,1
872642,1097544,412291,0.704641,7829,2
872642,1097544,412291,0.704641,9996,3
99506,1097512,984690,1.0,10440,1
99506,1097512,984690,1.0,3734,2
...,...,...,...,...,...
374802,119,591119,0.999797,4436,2
374808,119,123470,0.996359,7626,3
374808,119,123470,0.996359,3402,4
15433,107,507982,1.0,2657,1


In [94]:
not_enough_dict = recs[recs['user_id'].isin(s[s<N].index)].groupby('user_id')['item_id'].agg(lambda row: list(row)).to_dict()

In [95]:
# Creating dataset for PopularModel
dataset_train = Dataset.construct(
    interactions_df=train,
    user_features_df=None,
    item_features_df=None
)

In [96]:
popular_model = PopularModel()
popular_model.fit(dataset_train)

In [97]:
popular_model_result = popular_model.recommend(
    dataset_train.user_id_map.external_ids[:1], 
    dataset=dataset_train, 
    k=2*N, 
    filter_viewed=False,
)['item_id'].to_list()

In [98]:
d = {"user_id":[], "item_id": [], "rank": []}
# for user_id, max_rank in not_enough_dict.items():
for user_id, item_id_list in not_enough_dict.items():
    max_rank = len(item_id_list)
    # drop popular items, that is already in recommendation list
    corrected_popular_list = [item for item in popular_model_result if item not in item_id_list]
    for i in range(1, N - max_rank + 1):
        d['user_id'].append(user_id)
        d['item_id'].append(corrected_popular_list[i])
        d['rank'].append(max_rank+i)

In [99]:
filled_with_popular_CR_20 = pd.DataFrame.from_dict(d)

In [101]:
final_result_CR_20 = pd.concat([recs,filled_with_popular_CR_20])

In [102]:
with open('/content/drive/MyDrive/MTS_RecSys/final_result_CR_20.dill', 'wb') as f:
    dill.dump(final_result_CR_20, f)

#### TF_IDF_15

In [103]:
mapper = generate_implicit_recs_mapper(
    TFIDF_15, 
    N=N,
    users_mapping=users_mapping,
    users_inv_mapping=users_inv_mapping
)

In [104]:
recs = pd.DataFrame({
    'user_id': test['user_id'].unique()
})

recs['similar_user_id'], recs['similarity'] = zip(*recs['user_id'].map(mapper))
recs.head()

,user_id,similar_user_id,similarity
0,1016458,"[1016458, 542443, 983874, 263122, 1087135, 299198, 416417, 107556, 980255, 55914]","[0.9999999999999997, 0.30795056944195065, 0.2892556288628258, 0.2865786318030271, 0.2840258603538891, 0.28275509313396374, 0.2814949055823506, 0.2806520562356141, 0.27569737451648724, 0.2703263699..."
1,68478,"[68478, 620526, 695810, 20161, 412019, 879494, 589849, 988152, 691867, 1059229]","[0.9999999999999998, 0.6879105673985805, 0.6879105673985805, 0.6879105673985805, 0.6879105673985805, 0.6875684365910972, 0.682527643497476, 0.6810886673793897, 0.6649011119333184, 0.6500976114158797]"
2,580093,"[580093, 805265, 980409, 939663, 708439, 736490, 943049, 86046, 522603, 521456]","[1.0, 0.7937798109102163, 0.788744014339269, 0.7666797469287155, 0.7631606741401563, 0.750189495128039, 0.7501195711404515, 0.7474499190666032, 0.7473880080188159, 0.7471532852053147]"
3,1072552,"[1072552, 162216, 613583, 896269, 449057, 246357, 992547, 49193, 503934, 639316]","[1.0000000000000002, 0.45699036174245683, 0.45472129140483386, 0.4278037012555298, 0.42654520594852924, 0.42378450328959727, 0.41383525556374756, 0.411008423568197, 0.4078683617582729, 0.407690880..."
4,910002,"[910002, 301452, 454746, 827646, 331461, 105556, 1048599, 854405, 15649, 25643]","[0.9999999999999997, 0.5504875268884805, 0.5503635062149785, 0.5497829629361086, 0.5497829629361086, 0.5497829629361086, 0.5497829629361086, 0.5497829629361086, 0.5497829629361086, 0.5497829629361..."


In [105]:
# Exploding lists to get vertical representation
recs = recs.set_index('user_id').apply(pd.Series.explode).reset_index()

In [106]:
# Deleting recommendations of itself 
recs = recs[recs.user_id!=recs.similar_user_id]

In [107]:
watched = train.groupby('user_id').agg({'item_id': list})
watched.head()

,item_id
user_id,
0,"[7102, 14359, 15297, 6006, 9728, 12192]"
2,"[7571, 3541, 15266, 13867, 12841, 10770, 4475, 9506, 8936, 11018, 11577, 561, 7106, 6774, 16029, 8482, 6825, 3594, 16166, 5819, 2954, 383, 11689, 12449, 2025, 6155, 3628, 334, 4024, 7210, 11539, 1..."
3,"[9728, 16406, 10440, 3475, 4151, 1418, 2220, 3734, 13789, 8581, 13849, 9550, 10464, 4880, 4436, 8801, 11790, 2657, 8252]"
4,"[4700, 6317]"
5,"[14397, 6445, 11437, 5651, 6167, 12466, 632, 8450, 2685, 7825, 3145, 7043, 4179, 15890, 5115, 4719, 10848]"


In [108]:
# Joining watched items
recs = recs.merge(watched, left_on=['similar_user_id'], right_on=['user_id'], how='left')
recs = recs.explode('item_id')

In [109]:
# Dropping duplicates pairs user_id-item_id and keeping rows with the highest similiarity
recs = recs.sort_values(['user_id', 'similarity'], ascending=False)

In [110]:
recs = recs.drop_duplicates(['user_id', 'item_id'], keep='first')

In [111]:
# Making rank
recs['rank'] = recs.groupby('user_id').cumcount() + 1 

In [112]:
# Decreasing rank to N
recs = recs[recs['rank'] <=N]

In [113]:
# We have users with less than N recommendations
s = recs['user_id'].value_counts()
s[s<N]

556008    9
365738    9
137343    9
553069    9
231048    9
         ..
389271    1
389317    1
389370    1
389378    1
387893    1
Name: user_id, Length: 58932, dtype: int64

In [114]:
recs[recs['user_id'].isin(s[s<N].index)]

,user_id,similar_user_id,similarity,item_id,rank
99494,1097512,984690,0.999999,10440,1
99494,1097512,984690,0.999999,3734,2
603954,1097479,392250,0.849606,3734,1
603954,1097479,392250,0.849606,10770,2
603955,1097479,185082,0.838868,13865,3
...,...,...,...,...,...
849395,98,1002728,0.522362,10724,3
849396,98,978382,0.489714,4946,4
849396,98,978382,0.489714,4740,5
849397,98,243558,0.488654,2909,6


In [115]:
not_enough_dict = recs[recs['user_id'].isin(s[s<N].index)].groupby('user_id')['item_id'].agg(lambda row: list(row)).to_dict()

In [116]:
d = {"user_id":[], "item_id": [], "rank": []}
# for user_id, max_rank in not_enough_dict.items():
for user_id, item_id_list in not_enough_dict.items():
    max_rank = len(item_id_list)
    # drop popular items, that is already in recommendation list
    corrected_popular_list = [item for item in popular_model_result if item not in item_id_list]
    for i in range(1, N - max_rank + 1):
        d['user_id'].append(user_id)
        d['item_id'].append(corrected_popular_list[i])
        d['rank'].append(max_rank+i)

In [117]:
filled_with_popular_TFIDF_15 = pd.DataFrame.from_dict(d)

In [118]:
final_result_TFIDF_15 = pd.concat([recs,filled_with_popular_TFIDF_15])

In [119]:
with open('/content/drive/MyDrive/MTS_RecSys/final_result_TFIDF_15.dill', 'wb') as f:
    dill.dump(final_result_TFIDF_15, f)

##2.3. Blending

In [120]:
with open('/content/drive/MyDrive/MTS_RecSys/final_result_CR_20.dill', 'rb') as f:
    final_result_CR_20 = dill.load(f)

In [121]:
with open('/content/drive/MyDrive/MTS_RecSys/final_result_TFIDF_15.dill', 'rb') as f:
    final_result_TFIDF_15 = dill.load(f)

In [122]:
model1_result = final_result_tfidf[['user_id', 'item_id', 'rank']]
model2_result = final_result_userknn[['user_id', 'item_id', 'rank']]

In [123]:
popular_model_full_result = popular_model.recommend(
    dataset_train.user_id_map.external_ids[:1], 
    dataset=dataset_train, 
    k=train.item_id.nunique(), 
    filter_viewed=False,
)[['item_id','rank']]

In [124]:
merged_results = pd.concat([model1_result, model2_result])

In [125]:
from collections import Counter

# We will put items in the intersection on the top, 
# the rest of the items we will sort by popularity
def blend_items(row, N):
    item_list = list(row)
    cnt = Counter(item_list)
    top_items = [k for k, v in cnt.items() if v > 1]
    other_items = [k for k, v in cnt.items() if v == 1]
    items_to_add = popular_model_full_result[popular_model_full_result['item_id'].isin(other_items)]['item_id'].to_list()
    result = top_items + items_to_add[:N - len(top_items)] # to get N recommendations total
    return result
    

In [126]:
%%time
N = 10
resorted_results = merged_results.groupby('user_id')['item_id'].agg(lambda row: blend_items(row, N))

CPU times: user 54.5 s, sys: 157 ms, total: 54.7 s
Wall time: 55.7 s


In [127]:
resorted_results

user_id
3           [10440, 15297, 9728, 4151, 3734, 2657, 4880, 9996, 11640, 14461]
14             [10440, 13865, 9728, 4151, 3734, 2657, 4880, 142, 6809, 9996]
30             [13865, 9728, 676, 429, 9381, 16197, 4024, 2612, 14581, 2816]
47           [7401, 12965, 496, 11109, 10440, 13865, 6809, 4495, 1449, 5411]
53         [10440, 15297, 4880, 7107, 16509, 3509, 16107, 8211, 6607, 12418]
                                         ...                                
1097508      [8157, 12225, 7592, 8636, 3182, 11985, 10323, 14120, 8209, 973]
1097512        [10440, 3734, 9728, 4151, 2657, 4880, 142, 6809, 9996, 15297]
1097513       [3734, 4457, 4436, 15464, 5693, 10772, 8801, 9937, 1776, 2780]
1097534     [4880, 9996, 2657, 10440, 15297, 9728, 4151, 12192, 14431, 5658]
1097544       [9728, 15297, 13865, 4151, 3734, 2657, 4880, 9996, 8636, 7829]
Name: item_id, Length: 101982, dtype: object

In [128]:
final_blended_result = resorted_results.reset_index().explode('item_id')
final_blended_result['rank'] = final_blended_result.groupby('user_id').cumcount() + 1

In [129]:
final_blended_result

,user_id,item_id,rank
0,3,10440,1
0,3,15297,2
0,3,9728,3
0,3,4151,4
0,3,3734,5
...,...,...,...
101981,1097544,2657,6
101981,1097544,4880,7
101981,1097544,9996,8
101981,1097544,8636,9


In [130]:
with open('/content/drive/MyDrive/MTS_RecSys/blended_result.dill', 'wb') as f:
    dill.dump(final_blended_result, f)

### Metrics

In [131]:
with open('/content/drive/MyDrive/MTS_RecSys/blended_result.dill', 'rb') as f:
    blended_result = dill.load(f)

In [132]:
# Calculating several metrics, including MAP@10
metrics = {
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
    "map@10": MAP(k=10),
}

In [133]:
catalog = train['item_id'].unique()

In [134]:
blended_metric_values = calc_metrics(
      metrics,
      reco=blended_result,
      interactions=test,
      prev_interactions=train,
      catalog=catalog
)

In [135]:
blended_metric_values

{'prec@10': 0.02210466450290735,
 'recall@10': 0.1137896078784046,
 'map@10': 0.03180166994166452,
 'novelty': 5.149389238671356,
 'serendipity': 2.9223631049885478e-05}

## 2.4.Recommendations for cold users

In [ ]:
# Creating dataset for PopularModel
dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=None,
    item_features_df=None
)

In [ ]:
popular_model = PopularModel()
popular_model.fit(dataset)

In [ ]:
popular_model_result = popular_model.recommend(
    dataset.user_id_map.external_ids[:1], 
    dataset=dataset, 
    k=N, 
    filter_viewed=False,
)['item_id'].to_list()

In [ ]:
popular_model_result

[10440, 15297, 9728, 13865, 4151, 3734, 2657, 4880, 142, 6809]

In [ ]:
with open('/content/drive/MyDrive/MTS_RecSys/popular_items.dill', 'wb') as f:
    dill.dump(popular_model_result, f)
     

# 3.Inference online model

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import requests
from tqdm.auto import tqdm
from scipy.stats import mode 
from pprint import pprint
import dill
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender, BM25Recommender
import warnings
warnings.filterwarnings("ignore")

from rectools import Columns
from rectools.model_selection import TimeRangeSplitter
from rectools.dataset import Dataset, Interactions
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics, MAP
from rectools.models.popular import PopularModel
from collections import Counter


pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

In [2]:
interactions = pd.read_csv('kion_train/interactions.csv')

## Auxiliary functions

In [3]:
def get_coo_matrix(df, 
                   user_col='user_id', 
                   item_col='item_id', 
                   weight_col=None, 
                   users_mapping=None, 
                   items_mapping=None):
    if weight_col:
        weights = df[weight_col].astype(np.float32)
    else:
        weights = np.ones(len(df), dtype=np.float32)

    interaction_matrix = sp.sparse.coo_matrix((
        weights, 
        (
            df[user_col].map(users_mapping.get), 
            df[item_col].map(items_mapping.get)
        )
    ))
    return interaction_matrix

In [4]:
def generate_implicit_recs_mapper(model, N, users_mapping, users_inv_mapping):
    def _recs_mapper(user):
        user_id = users_mapping[user]
        recs = model.similar_items(user_id, N=N)
        return [users_inv_mapping[user] for user, _ in recs], [sim for _, sim in recs]
    return _recs_mapper

In [5]:
# Renaming columns, converting timestamp
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight}, 
                    inplace=True) 

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

In [6]:
users_inv_mapping = dict(enumerate(interactions['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}

In [7]:
items_inv_mapping = dict(enumerate(interactions['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}

In [8]:
print(f"users_mapping amount: {len(users_mapping)}")
print(f"items_mapping amount: {len(items_mapping)}")

users_mapping amount: 962179
items_mapping amount: 15706


Load trained model

In [9]:
with open('/content/drive/MyDrive/MTS_RecSys/TFIDF_15.dill', 'rb') as f:
    model = dill.load(f)

In [10]:
model

In [11]:
N=10

## Preparation

In [12]:
# Creating dataset for PopularModel
dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=None,
    item_features_df=None
)

In [13]:
popular_model = PopularModel()
popular_model.fit(dataset)

In [14]:
popular_model_result = popular_model.recommend(
    dataset.user_id_map.external_ids[:1], 
    dataset=dataset, 
    k=N, 
    filter_viewed=False,
)['item_id'].to_list()

In [15]:
popular_model_result

[10440, 15297, 9728, 13865, 4151, 3734, 2657, 4880, 142, 6809]

## Execution

In [16]:
user_id = 1097479

In [17]:
if any(interactions['user_id']==user_id):
    mapper = generate_implicit_recs_mapper(
        model, 
        N=N,
        users_mapping=users_mapping,
        users_inv_mapping=users_inv_mapping
    )
    recs = pd.DataFrame({
        'user_id': [user_id]
    })
    recs['similar_user_id'], recs['similarity'] = zip(*recs['user_id'].map(mapper))

    # Exploding lists to get vertical representation
    recs = recs.set_index('user_id').apply(pd.Series.explode).reset_index()
    # Deleting recommendations of itself   
    recs = recs[~(recs['user_id']==recs['similar_user_id'])]

    # Joining watched items
    watched = interactions.groupby('user_id').agg({'item_id': list})
    recs = recs.merge(watched, left_on=['similar_user_id'], right_on=['user_id'], how='left')
    recs = recs.explode('item_id')

    # Dropping duplicates pairs user_id-item_id and keeping rows with the highest similiarity
    recs = recs.sort_values(['user_id', 'similarity'], ascending=False)
    recs = recs.drop_duplicates(['user_id', 'item_id'], keep='first')

    # Making rank
    recs['rank'] = recs.groupby('user_id').cumcount() + 1 

    # Decreasing rank to 10
    recs = recs[recs['rank'] <= 10]

    # Dealing witn less than N recommendations
    s = recs['user_id'].value_counts()
    not_enough_dict = recs[recs['user_id'].isin(s[s<N].index)].groupby('user_id')['item_id'].agg(lambda row: list(row)).to_dict()

    d = {"user_id":[], "item_id": [], "rank": []}
    # for user_id, max_rank in not_enough_dict.items():
    for user_id, item_id_list in not_enough_dict.items():
        max_rank = len(item_id_list)
        # drop popular items, that is already in recommendation list
        corrected_popular_list = [item for item in popular_model_result if item not in item_id_list]
        for i in range(1, N - max_rank + 1):
            d['user_id'].append(user_id)
            d['item_id'].append(corrected_popular_list[i])
            d['rank'].append(max_rank+i)

    filled_with_popular = pd.DataFrame.from_dict(d)
    final_result = pd.concat([recs, filled_with_popular])

    print(final_result.item_id.to_list())
else:
    # recommend for cold user
    print(popular_model_result)

[2657, 15531, 3495, 372, 12537, 14159, 10219, 6660, 12192, 2720]
